# LFT: From Three Fundamental Laws to $A = L(I)$

**Goal.** Make explicit how the Three Fundamental Laws (Identity, Non-Contradiction, Excluded Middle) become a *generative operator* $L$ that filters the information space $I$ into actuality $A$.

We keep this notebook **dependency-free** and illustrative. Full-scale implementations (graphs, Cayley structures, permutohedra) appear in Notebooks 01–06.

## 1. Three Fundamental Laws (3FLL)
We model patterns as directed relations (ordered pairs) on a finite label set $E=\{0,\dots,N-1\}$. The laws act as constraints:

1. **Identity (ID)** — *Persistence / Reflexivity.*  
   Require $(i,i)$ for all $i\in E$. Intuition: self-identity across updates.

2. **Non-Contradiction (NC)** — *Acyclicity.*  
   Forbid directed cycles in the non-reflexive edges. Intuition: no contradictory preference loops.

3. **Excluded Middle (EM)** — *Decisiveness / Totalization when justified.*  
   When a relation is globally consistent (acyclic and definable), complete it to a **total order** (linear extension) if sufficient information exists.

These laws compose to a logical operator $L$.

## 2. Operator Composition
We define the **logical filter**:

$$ L \,=\, \mathrm{EM}\circ\mathrm{NC}\circ\mathrm{ID}, \qquad A = L(I). $$

- $\mathrm{ID}$: insert/preserve all reflexive edges.
- $\mathrm{NC}$: reject (map to $\bot$) any pattern with cycles.
- $\mathrm{EM}$: when feasible, complete to a total order; otherwise leave as a partial order.

Later notebooks implement $L$ *locally*, but this global composition captures the essence.

## 3. Minimal Algorithmic Schema (pure Python)
We provide a compact, dependency-free implementation that:

1. Adds reflexive edges (ID),
2. Checks for cycles (NC),
3. Attempts totalization (EM) by topological sorting when the edge set is complete for a total order.

> For clarity we separate **partial-order acceptance** from **totalization**; EM only totalizes when the number of non-reflexive edges equals $\binom{N}{2}$.

In [ ]:
from typing import List, Tuple, Optional, Set
import os

# Ensure outputs directory exists
os.makedirs('./outputs', exist_ok=True)

Edge = Tuple[int,int]

def has_cycle(n:int, edges:Set[Edge]) -> bool:
    """DFS-based cycle detection on directed graph with nodes 0..n-1."""
    adj = {i:[] for i in range(n)}
    for a,b in edges:
        if a!=b:
            adj[a].append(b)
    state = [0]*n  # 0=unseen,1=visiting,2=done
    def dfs(u:int) -> bool:
        state[u]=1
        for v in adj[u]:
            if state[v]==1:
                return True
            if state[v]==0 and dfs(v):
                return True
        state[u]=2
        return False
    for i in range(n):
        if state[i]==0 and dfs(i):
            return True
    return False

def topo_sort_if_possible(n:int, edges:Set[Edge]) -> Optional[List[int]]:
    """Return one topological order if DAG, else None."""
    indeg = [0]*n
    adj = {i:[] for i in range(n)}
    for a,b in edges:
        if a!=b:
            adj[a].append(b); indeg[b]+=1
    order = []
    q = [i for i in range(n) if indeg[i]==0]
    while q:
        u = q.pop(0)
        order.append(u)
        for v in adj[u]:
            indeg[v]-=1
            if indeg[v]==0:
                q.append(v)
    return order if len(order)==n else None

def count_linear_extensions(n:int, edges:Set[Edge], limit:int=10) -> int:
    """Count number of linear extensions (total orders) of a DAG."""
    adj = {i:[] for i in range(n)}
    indeg = [0]*n
    for a,b in edges:
        if a!=b:
            adj[a].append(b); indeg[b]+=1
    
    def backtrack(current_order, remaining_indeg):
        if len(current_order) == n:
            return 1
        count = 0
        available = [i for i in range(n) if i not in current_order and remaining_indeg[i] == 0]
        for node in available:
            if count >= limit:  # Prevent exponential blowup
                return limit
            new_indeg = remaining_indeg[:]
            for neighbor in adj[node]:
                new_indeg[neighbor] -= 1
            count += backtrack(current_order + [node], new_indeg)
        return count
    
    return backtrack([], indeg[:])

def apply_L(n:int, nonref:List[Edge]):
    """Apply ID -> NC -> EM. Returns (status, pattern, order).
    status in { 'reject', 'partial', 'total' }.
    pattern is the reflexive-augmented edge set if accepted; order is a list if total.
    """
    # ID: Add reflexive edges
    pattern = set(nonref) | {(i,i) for i in range(n)}
    
    # NC: Check for cycles
    if has_cycle(n, pattern):
        return 'reject', None, None
    
    # EM: Enhanced logic - check if unique linear extension exists
    nonref_only = {(a,b) for (a,b) in pattern if a!=b}
    
    # Count linear extensions to determine if EM should totalize
    num_extensions = count_linear_extensions(n, nonref_only)
    
    if num_extensions == 1:
        # Unique total order exists - EM totalizes
        order = topo_sort_if_possible(n, nonref_only)
        return 'total', pattern, order
    elif num_extensions == 0:
        # Impossible case (should be caught by NC)
        return 'reject', None, None
    else:
        # Multiple extensions possible - remains partial
        return 'partial', pattern, None

# Test suite for validation
print("Testing Logical Operator L Implementation")
print("=======================================")

test_cases = [
    # Test 1: Simple partial order
    (4, [(0,1), (1,2)], 'partial', "Simple chain - multiple extensions"),
    
    # Test 2: Complete total order  
    (4, [(0,1), (1,2), (2,3), (0,2), (1,3), (0,3)], 'total', "Complete total order"),
    
    # Test 3: Cycle rejection
    (3, [(0,1), (1,2), (2,0)], 'reject', "Directed cycle"),
    
    # Test 4: Empty relation (all extensions possible)
    (3, [], 'partial', "Empty relation - many extensions"),
    
    # Test 5: Single edge
    (3, [(0,1)], 'partial', "Single constraint"),
]

results = []
for i, (n, edges, expected, description) in enumerate(test_cases):
    status, pattern, order = apply_L(n, edges)
    passed = status == expected
    results.append((i+1, description, status, expected, passed))
    
    print(f"Test {i+1}: {description}")
    print(f"  Input: n={n}, edges={edges}")
    print(f"  Result: {status} (expected: {expected}) {'✓' if passed else '✗'}")
    if status == 'total':
        print(f"  Total order: {order}")
    print()

all_passed = all(r[4] for r in results)
print(f"All tests passed: {all_passed}")

# Demonstrate the three laws in action
print("\nDemonstration of Three Fundamental Laws:")
print("======================================")

n = 4
demo_edges = [(0,1), (1,2), (0,2)]

# Step by step application
print(f"Input edges: {demo_edges}")

# ID: Add reflexive edges
with_id = set(demo_edges) | {(i,i) for i in range(n)}
print(f"After ID (add reflexive): {sorted(with_id)}")

# NC: Check acyclicity  
is_acyclic = not has_cycle(n, with_id)
print(f"After NC (acyclicity check): {'PASS' if is_acyclic else 'REJECT'}")

if is_acyclic:
    # EM: Check for unique extension
    nonref = {(a,b) for (a,b) in with_id if a!=b}
    extensions = count_linear_extensions(n, nonref, limit=5)
    print(f"After EM (extension count): {extensions} linear extensions")
    if extensions == 1:
        order = topo_sort_if_possible(n, nonref)
        print(f"Unique total order: {order}")
    else:
        print("Multiple extensions - remains partial order")

## 4. Correctness Intuitions and Validation

### Logical Flow Analysis:
- **ID → NC:** Reflexivity ensures stable identity; NC removes contradictions (cycles)  
- **NC → EM:** In a DAG, EM checks if exactly one linear extension exists → unique totalization
- **Enhanced EM Logic:** Unlike the simple "complete edge set" check, we now count actual linear extensions

### Outcomes Explained:
- **reject:** violates NC (cycles), or EM fails due to inconsistency
- **partial:** consistent but multiple linear extensions possible (EM cannot decide)  
- **total:** consistent and unique linear extension exists → corresponds to a **permutation**

### Key Insight: 
The enhanced EM implementation correctly identifies when a partial order has a unique completion, making the operator more precise than simple edge counting.

### Connection to Permutation Geometry:
Each 'total' output corresponds to a permutation of {0,1,...,n-1}, which will map to vertices of the permutohedron in the geometric realization (notebook 04_Geometry).

## 5. Why this yields permutation geometry (preview)
Applying $L$ to all patterns picks out **acyclic totals** which correspond one-to-one with permutations (linear extensions).  
Thus $S_N$ and its **$N-1$ adjacent generators** (Coxeter type $A_{N-1}$) appear *inevitably* from the logical program—not as an external ansatz.  

Notebook **02_Geometry** formalizes the sum-zero space $V\cong\mathbb{R}^{N-1}$ and the **permutohedron** $\Pi_{N-1}$, showing **rank = $N-1$** as intrinsic dimension.

## 6. Hand-off to later notebooks
- **01_Introduction**: enumeration at small $N$ and filtering statistics (cycles/partials/totals).
- **02_Geometry**: $A_{N-1}$ geometry; permutohedra and adjacent-edge Cayley graphs.
- **03_Dynamics\_Time**: inversion count $h$ as Lyapunov; time as $L$-flow.
- **04_Quantum\_Link**: simplex orbits and the qudit bridge.
- **05–06**: scaling to $N=6$ and spacetime factorization.